In [62]:
import numpy as np 
import matplotlib.pyplot as plt 
from weightedCSsequential import * 

In [63]:
A = 0.01
N = 200
N1 = 100
inv_prop = False
N2 = N-N1
f_over_S_range = [1-A, 1+A]

if inv_prop:
    title = f'{N1/N:.0%} large ' +r'$\pi$ values,    $f \propto 1/\pi$'
    figname = f'../data/NoSideInfoCSf_inv_propto_M_large_{N1}.png'
    f_ranges = [[1e-3, 2*1e-3], [0.4, 0.5]]
else:
    title = f'{N1/N:.0%} large ' +r'$\pi$ values,    $f \propto \pi$'
    figname = f'../data/NoSideInfoCSf_propto_M_large_{N1}.png'
    f_ranges = [[0.4, 0.5], [1e-3, 2*1e-3]]



M, f, S =  generate_MFS(N_vals=(N1, N2), 
            N=N, # total number of transactions = sum(N_vals)
            M_ranges = [ [1e5, 1e6], [1e2, 1*1e3]],  
            f_ranges = f_ranges, 
            a = A)

m_star = np.sum(M*f)/ np.sum(M)

In [64]:
nG = 20
grid = np.linspace(0, 1, nG) 
grid = np.append(grid, [m_star])
nG += 1
wealth = np.ones((nG,))
Wealth = np.ones((nG,1))
Payoff_vals = np.zeros((nG,1))
seen = None # indicies queried so far 
unseen = np.arange(N) # indicies not queried so far 
Transaction_Indices = np.zeros((N,))
cv_vals=None
beta_vals = None
use_CV = False 
method_name = 'propMS'
beta_max = 0.5
lambda_max = 4
alpha=0.05
Lambda_Vals = np.zeros((nG, N))


result = run_one_expt(M, f,  S, method_name='propMS', lambda_max=lambda_max,
                    beta_max=0.5, nG=nG, use_CV=False, 
                    f_over_S_range=None, alpha=0.05, logical_CS=False, 
                    intersect=False, return_payoff=True)

In [65]:
grid, Wealth, LowerCS, UpperCS, Transaction_Indices, Error_flag, Payoff_vals = result


In [66]:
def manual_payoffs(m, M, f, S, Idx, f_over_S,  method='propMS'):
    Idx = Idx.astype(int)
    M_, f_, S_ = M[Idx], f[Idx], S[Idx]
    MS_U = np.flip(np.cumsum(np.flip(M_*S_)))
    M_sum = np.sum(M_)
    temp = MS_U / M_sum 
    Mu0 = np.zeros((len(M),))
    Mu0[1:] = np.cumsum(M_[:-1]*f_[:-1])/ M_sum
    if method=='propMS':
        payoff = temp*f_/S_ - m + Mu0

    L, U = np.zeros((N,)), np.zeros((N,))
    for t in range(N):
        unseen = Idx[t:]
        mu_0 = Mu0[t]
        q_t = get_sampling_distribution2(unseen, M, f, S, method)
        L[t], U[t] = get_range_of_next_payoff(m, unseen, q_t, mu_0, M, S, method_name=method, 
                                f_over_S_range=None, use_CV=False, beta=0.5)
    
    return payoff, L, U
        

In [67]:
for i, _ in enumerate(grid):
    m = grid[i]
    P, L, U = manual_payoffs(m, M, f, S, Transaction_Indices, f_over_S_range)

    a = P[P>U]
    b = P[P<L]
    if len(a) or len(b):
        print(f'{i}: \t {len(a)}, {len(b)}')
